In [1]:
using Distributed; addprocs(5, exeflags="--project")
using DelimitedFiles

In [2]:
# Base functionality
@everywhere begin
    using Plots, Measures, Statistics

    # ODE Solver
    using DifferentialEquations

    # Benchmarking
    using BenchmarkTools

    include("../MixedMealModel.jl")
end

In [3]:
@everywhere function fit_individual(individual)
    glc, ins, trg, nfa, bwg, time = SampleData()
    model, constants, subject = ModelFromData(glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], bwg[individual])
    loss, simulationfunc = LossFunction(model, glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], Int.(time), constants, subject)
    #p_init = [1.35e-2, 3.80e-3, 5.82e-1, 0.00045, 0.0713, 208.88, 0.0163, 0.0119]
    p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
    p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]
    println("Fitting individual $(individual)")
    #println("Initial loss for individual $(individual): $(loss(p_init))")
    res, obj = FitModelLHC(loss, 5, p_lb, p_ub)
    return res, obj
end

In [4]:
@everywhere function FitAllIndividuals()
    @sync @distributed for i = 1:5
        fit_individual(i)
    end
end


In [5]:
BenchmarkResult = @benchmark FitAllIndividuals() samples=100 seconds=60*60*3

      From worker 4:	Fitting individual 3
      From worker 6:	Fitting individual 5
      From worker 5:	Fitting individual 4
      From worker 2:	Fitting individual 1
      From worker 3:	Fitting individual 2


      From worker 3:	Optimization Failed... Resampling...


      From worker 5:	Fitting individual 4
      From worker 2:	Fitting individual 1
      From worker 4:	Fitting individual 3
      From worker 6:	Fitting individual 5
      From worker 3:	Fitting individual 2


      From worker 2:	Fitting individual 1
      From worker 5:	Fitting individual 4
      From worker 3:	Fitting individual 2
      From worker 4:	Fitting individual 3
      From worker 6:	Fitting individual 5


In [ ]:
BenchmarkResult

In [ ]:
open("Windows_ShitMachine_Benchmark_ParamEstim.txt", "w+") do io
    writedlm(io, BenchmarkResult.times)
end